URL file model h5 : https://drive.google.com/file/d/1JUsB8qG6c_XMzQqIV97WF40HWDBd5Yue/view?usp=drive_link

In [61]:
import pandas as pd
import numpy as np

In [62]:
df = pd.read_csv('(main) book_data.csv')

In [63]:
df.head(2)

,books_id,title,url_playbook,url_image,synopsis,idr,isbn,author,avg_rating,tags1,tags2,tags3
0,1,Harry Potter and the Half-Blood Prince by J.K....,https://play.google.com/store/books/details/Br...,https://books.google.com/books/publisher/conte...,<b>Unlock the more straightforward side of <i>...,"$81,056.00",439785960,"J.K. Rowling, Mary GrandPrÃ©","4,54",fantasy,fiction,young-adult
1,2,Harry Potter and the Order of the Phoenix by J...,https://play.google.com/store/books/details/Br...,https://books.google.com/books/publisher/conte...,<b>Unlock the more straightforward side of <i>...,"$81,056.00",439358078,"J.K. Rowling, Mary GrandPrÃ©","4,46",children,children-s,fantasy


In [64]:
df.shape

(6205, 12)

In [65]:
df.isnull().sum()

books_id          0
title             0
url_playbook      0
url_image         0
synopsis         75
idr               0
isbn              0
author          339
avg_rating        0
tags1             0
tags2             0
tags3             0
dtype: int64

In [66]:
df= df.dropna()

In [67]:
df.isnull().sum()

books_id        0
title           0
url_playbook    0
url_image       0
synopsis        0
idr             0
isbn            0
author          0
avg_rating      0
tags1           0
tags2           0
tags3           0
dtype: int64

In [68]:
df.duplicated().sum()

0

In [69]:
df["genre"] = df["tags1"].astype(str) + ", " + df["tags2"].astype(str) + ", " + df["tags3"].astype(str)
df

,books_id,title,url_playbook,url_image,synopsis,idr,isbn,author,avg_rating,tags1,tags2,tags3,genre
0,1,Harry Potter and the Half-Blood Prince by J.K....,https://play.google.com/store/books/details/Br...,https://books.google.com/books/publisher/conte...,<b>Unlock the more straightforward side of <i>...,"$81,056.00",439785960,"J.K. Rowling, Mary GrandPrÃ©","4,54",fantasy,fiction,young-adult,"fantasy, fiction, young-adult"
1,2,Harry Potter and the Order of the Phoenix by J...,https://play.google.com/store/books/details/Br...,https://books.google.com/books/publisher/conte...,<b>Unlock the more straightforward side of <i>...,"$81,056.00",439358078,"J.K. Rowling, Mary GrandPrÃ©","4,46",children,children-s,fantasy,"children, children-s, fantasy"
2,3,"""Harry Potter and the Sorcerer's Stoneâ„¢"" -- ...",https://play.google.com/store/books/details/Jo...,https://books.google.com/books/publisher/conte...,This instrumental series contains selected the...,"$97,297.00",439554934,"J.K. Rowling, Mary GrandPrÃ©","4,44",fantasy,fiction,young-adult,"fantasy, fiction, young-adult"
3,5,Harry Potter and the Prisoner of Azkaban (Nove...,https://play.google.com/store/books/details/Ke...,https://books.google.com/books/publisher/conte...,Harry Potter and the Prisoner of Azkaban is ab...,"$145,126.00",439655480,"J.K. Rowling, Mary GrandPrÃ©, Rufus Beck","4,53",fantasy,fiction,young-adult,"fantasy, fiction, young-adult"
4,6,Harry Potter and the Goblet of Fire by J.K. Ro...,https://play.google.com/store/books/details/Br...,https://books.google.com/books/publisher/conte...,<b>Unlock the more straightforward side of <i>...,"$81,056.00",439139600,"J.K. Rowling, Mary GrandPrÃ©","4,53",fantasy,fiction,young-adult,"fantasy, fiction, young-adult"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6197,30008702,The 16 Personality Types in a Nutshell,https://play.google.com/store/books/details/Ja...,https://books.google.com/books/publisher/conte...,<b>Meet the self-sufficient and brilliant stra...,$0.00,385542070,Ian McEwan,"3,71",thriller,book-club,contemporary,"thriller, book-club, contemporary"
6198,30253864,Talking as Fast as I Can: From Gilmore Girls t...,https://play.google.com/store/books/details/La...,https://books.google.com/books/publisher/conte...,<b><i>NEW YORK TIMES</i> BESTSELLER â€¢ In thi...,"$148,851.00",425285170,Lauren Graham,"4,04",fiction,memoir,non-fiction,"fiction, memoir, non-fiction"
6200,30555488,The Underground Railroad: Winner of the Pulitz...,https://play.google.com/store/books/details/Co...,https://books.google.com/books/publisher/conte...,<b>WINNER OF THE PULITZER PRIZE FOR FICTION 20...,"$123,598.00",385542364,Colson Whitehead,"4,04",book-club,fiction,historical-fiction,"book-club, fiction, historical-fiction"
6203,31845516,Love Warrior: A Memoir,https://play.google.com/store/books/details/Gl...,https://books.google.com/books/publisher/conte...,<b>#1 <i>New York Times</i> Bestseller | </b><...,"$178,651.00",1250128544,Glennon Doyle Melton,"4,1",memoir,non-fiction,nonfiction,"memoir, non-fiction, nonfiction"


In [70]:
df['tag'] = df['title'] + df['synopsis'] + df['author'] + df['genre']
df

,books_id,title,url_playbook,url_image,synopsis,idr,isbn,author,avg_rating,tags1,tags2,tags3,genre,tag
0,1,Harry Potter and the Half-Blood Prince by J.K....,https://play.google.com/store/books/details/Br...,https://books.google.com/books/publisher/conte...,<b>Unlock the more straightforward side of <i>...,"$81,056.00",439785960,"J.K. Rowling, Mary GrandPrÃ©","4,54",fantasy,fiction,young-adult,"fantasy, fiction, young-adult",Harry Potter and the Half-Blood Prince by J.K....
1,2,Harry Potter and the Order of the Phoenix by J...,https://play.google.com/store/books/details/Br...,https://books.google.com/books/publisher/conte...,<b>Unlock the more straightforward side of <i>...,"$81,056.00",439358078,"J.K. Rowling, Mary GrandPrÃ©","4,46",children,children-s,fantasy,"children, children-s, fantasy",Harry Potter and the Order of the Phoenix by J...
2,3,"""Harry Potter and the Sorcerer's Stoneâ„¢"" -- ...",https://play.google.com/store/books/details/Jo...,https://books.google.com/books/publisher/conte...,This instrumental series contains selected the...,"$97,297.00",439554934,"J.K. Rowling, Mary GrandPrÃ©","4,44",fantasy,fiction,young-adult,"fantasy, fiction, young-adult","""Harry Potter and the Sorcerer's Stoneâ„¢"" -- ..."
3,5,Harry Potter and the Prisoner of Azkaban (Nove...,https://play.google.com/store/books/details/Ke...,https://books.google.com/books/publisher/conte...,Harry Potter and the Prisoner of Azkaban is ab...,"$145,126.00",439655480,"J.K. Rowling, Mary GrandPrÃ©, Rufus Beck","4,53",fantasy,fiction,young-adult,"fantasy, fiction, young-adult",Harry Potter and the Prisoner of Azkaban (Nove...
4,6,Harry Potter and the Goblet of Fire by J.K. Ro...,https://play.google.com/store/books/details/Br...,https://books.google.com/books/publisher/conte...,<b>Unlock the more straightforward side of <i>...,"$81,056.00",439139600,"J.K. Rowling, Mary GrandPrÃ©","4,53",fantasy,fiction,young-adult,"fantasy, fiction, young-adult",Harry Potter and the Goblet of Fire by J.K. Ro...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6197,30008702,The 16 Personality Types in a Nutshell,https://play.google.com/store/books/details/Ja...,https://books.google.com/books/publisher/conte...,<b>Meet the self-sufficient and brilliant stra...,$0.00,385542070,Ian McEwan,"3,71",thriller,book-club,contemporary,"thriller, book-club, contemporary",The 16 Personality Types in a Nutshell<b>Meet ...
6198,30253864,Talking as Fast as I Can: From Gilmore Girls t...,https://play.google.com/store/books/details/La...,https://books.google.com/books/publisher/conte...,<b><i>NEW YORK TIMES</i> BESTSELLER â€¢ In thi...,"$148,851.00",425285170,Lauren Graham,"4,04",fiction,memoir,non-fiction,"fiction, memoir, non-fiction",Talking as Fast as I Can: From Gilmore Girls t...
6200,30555488,The Underground Railroad: Winner of the Pulitz...,https://play.google.com/store/books/details/Co...,https://books.google.com/books/publisher/conte...,<b>WINNER OF THE PULITZER PRIZE FOR FICTION 20...,"$123,598.00",385542364,Colson Whitehead,"4,04",book-club,fiction,historical-fiction,"book-club, fiction, historical-fiction",The Underground Railroad: Winner of the Pulitz...
6203,31845516,Love Warrior: A Memoir,https://play.google.com/store/books/details/Gl...,https://books.google.com/books/publisher/conte...,<b>#1 <i>New York Times</i> Bestseller | </b><...,"$178,651.00",1250128544,Glennon Doyle Melton,"4,1",memoir,non-fiction,nonfiction,"memoir, non-fiction, nonfiction",Love Warrior: A Memoir<b>#1 <i>New York Times<...


In [71]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
# Function for removing NonAscii characters
def _removeNonAscii(s):
    return "".join(i for i in s if  ord(i)<128)
# Function for converting into lower case
def make_lower_case(text):
    return text.lower()
# Function for removing stop words
def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text
# Function for removing punctuation
def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text
#Function for removing the html tags
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)
# Applying all the functions in description and storing as a cleaned_desc
df['tags'] = df['tag'].apply(_removeNonAscii)
df['tags'] = df.tags.apply(func = make_lower_case)
df['tags'] = df.tags.apply(func = remove_stop_words)
df['tags'] = df.tags.apply(func=remove_punctuation)
df['tags'] = df.tags.apply(func=remove_html)


In [72]:
df

,books_id,title,url_playbook,url_image,synopsis,idr,isbn,author,avg_rating,tags1,tags2,tags3,genre,tag,tags
0,1,Harry Potter and the Half-Blood Prince by J.K....,https://play.google.com/store/books/details/Br...,https://books.google.com/books/publisher/conte...,<b>Unlock the more straightforward side of <i>...,"$81,056.00",439785960,"J.K. Rowling, Mary GrandPrÃ©","4,54",fantasy,fiction,young-adult,"fantasy, fiction, young-adult",Harry Potter and the Half-Blood Prince by J.K....,harry potter half blood prince j k rowling boo...
1,2,Harry Potter and the Order of the Phoenix by J...,https://play.google.com/store/books/details/Br...,https://books.google.com/books/publisher/conte...,<b>Unlock the more straightforward side of <i>...,"$81,056.00",439358078,"J.K. Rowling, Mary GrandPrÃ©","4,46",children,children-s,fantasy,"children, children-s, fantasy",Harry Potter and the Order of the Phoenix by J...,harry potter order phoenix j k rowling book an...
2,3,"""Harry Potter and the Sorcerer's Stoneâ„¢"" -- ...",https://play.google.com/store/books/details/Jo...,https://books.google.com/books/publisher/conte...,This instrumental series contains selected the...,"$97,297.00",439554934,"J.K. Rowling, Mary GrandPrÃ©","4,44",fantasy,fiction,young-adult,"fantasy, fiction, young-adult","""Harry Potter and the Sorcerer's Stoneâ„¢"" -- ...",harry potter sorcerer s stone selected themes ...
3,5,Harry Potter and the Prisoner of Azkaban (Nove...,https://play.google.com/store/books/details/Ke...,https://books.google.com/books/publisher/conte...,Harry Potter and the Prisoner of Azkaban is ab...,"$145,126.00",439655480,"J.K. Rowling, Mary GrandPrÃ©, Rufus Beck","4,53",fantasy,fiction,young-adult,"fantasy, fiction, young-adult",Harry Potter and the Prisoner of Azkaban (Nove...,harry potter prisoner azkaban novel study gr 4...
4,6,Harry Potter and the Goblet of Fire by J.K. Ro...,https://play.google.com/store/books/details/Br...,https://books.google.com/books/publisher/conte...,<b>Unlock the more straightforward side of <i>...,"$81,056.00",439139600,"J.K. Rowling, Mary GrandPrÃ©","4,53",fantasy,fiction,young-adult,"fantasy, fiction, young-adult",Harry Potter and the Goblet of Fire by J.K. Ro...,harry potter goblet fire j k rowling book anal...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6197,30008702,The 16 Personality Types in a Nutshell,https://play.google.com/store/books/details/Ja...,https://books.google.com/books/publisher/conte...,<b>Meet the self-sufficient and brilliant stra...,$0.00,385542070,Ian McEwan,"3,71",thriller,book-club,contemporary,"thriller, book-club, contemporary",The 16 Personality Types in a Nutshell<b>Meet ...,16 personality types nutshell b meet self suff...
6198,30253864,Talking as Fast as I Can: From Gilmore Girls t...,https://play.google.com/store/books/details/La...,https://books.google.com/books/publisher/conte...,<b><i>NEW YORK TIMES</i> BESTSELLER â€¢ In thi...,"$148,851.00",425285170,Lauren Graham,"4,04",fiction,memoir,non-fiction,"fiction, memoir, non-fiction",Talking as Fast as I Can: From Gilmore Girls t...,talking fast can gilmore girls gilmore girls a...
6200,30555488,The Underground Railroad: Winner of the Pulitz...,https://play.google.com/store/books/details/Co...,https://books.google.com/books/publisher/conte...,<b>WINNER OF THE PULITZER PRIZE FOR FICTION 20...,"$123,598.00",385542364,Colson Whitehead,"4,04",book-club,fiction,historical-fiction,"book-club, fiction, historical-fiction",The Underground Railroad: Winner of the Pulitz...,underground railroad winner pulitzer prize fic...
6203,31845516,Love Warrior: A Memoir,https://play.google.com/store/books/details/Gl...,https://books.google.com/books/publisher/conte...,<b>#1 <i>New York Times</i> Bestseller | </b><...,"$178,651.00",1250128544,Glennon Doyle Melton,"4,1",memoir,non-fiction,nonfiction,"memoir, non-fiction, nonfiction",Love Warrior: A Memoir<b>#1 <i>New York Times<...,love warrior memoir b 1 i new york times i bes...


In [73]:
df_fix = df[['books_id', 'title', 'tags']]

In [74]:
df_fix.head(5)

,books_id,title,tags
0,1,Harry Potter and the Half-Blood Prince by J.K....,harry potter half blood prince j k rowling boo...
1,2,Harry Potter and the Order of the Phoenix by J...,harry potter order phoenix j k rowling book an...
2,3,"""Harry Potter and the Sorcerer's Stoneâ„¢"" -- ...",harry potter sorcerer s stone selected themes ...
3,5,Harry Potter and the Prisoner of Azkaban (Nove...,harry potter prisoner azkaban novel study gr 4...
4,6,Harry Potter and the Goblet of Fire by J.K. Ro...,harry potter goblet fire j k rowling book anal...


In [75]:
from sklearn.feature_extraction.text import CountVectorizer
cv  = CountVectorizer(max_features=5000 , stop_words= 'english')

cv.fit_transform(df_fix['tags']).toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [76]:
vectors = cv.fit_transform(df_fix['tags']).toarray()

In [77]:
cv.get_feature_names_out()

array(['000', '10', '100', ..., 'zombie', 'zone', 'zoo'], dtype=object)

In [78]:
import nltk
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [79]:
def stem(text):
    y = []

    for i in text.split():
        y.append(ps.stem(i))
    return ' '.join(y)

In [80]:
stem('action') , stem('actionable'), stem('actioning') , stem('historical') , stem('historically')

('action', 'action', 'action', 'histor', 'histor')

In [81]:
df_fix['tags'][0]

'harry potter half blood prince j k rowling book analysis detailed summary analysis reading guide b unlock straightforward side i harry potter half blood prince i concise insightful summary analysis b br br this engaging summary presents analysis i harry potter half blood prince i j k rowling follows harry tries learn much possible lord voldemorts powers weaknesses still keeping schoolwork dealing teenage drama trying keep eye rival draco malfoy behaving oddly ever since return hogwarts i harry potter half blood prince i sixth novel j k rowlings bestselling seven book series inspired films video games board games even theme park rowling household name around world books include i the casual vacancy i 2012 i the cuckoos calling i 2013 published pseudonym robert galbraith br br find everything need know i harry potter half blood prince i in fraction time br br b i this in depth informative reading guide brings you i b br br complete plot summary br character studies br key themes symbols

In [82]:
stem('harry potter half blood prince j k rowling book analysis detailed summary analysis reading guide b unlock straightforward side i harry potter half blood prince i concise insightful summary analysis b br br this engaging summary presents analysis i harry potter half blood prince i j k rowling follows harry tries learn much possible lord voldemorts powers weaknesses still keeping schoolwork dealing teenage drama trying keep eye rival draco malfoy behaving oddly ever since return hogwarts i harry potter half blood prince i sixth novel j k rowlings bestselling seven book series inspired films video games board games even theme park rowling household name around world books include i the casual vacancy i 2012 i the cuckoos calling i 2013 published pseudonym robert galbraith br br find everything need know i harry potter half blood prince i in fraction time br br b i this in depth informative reading guide brings you i b br br complete plot summary br character studies br key themes symbols br questions reflection br br i why choose brightsummaries com i br br available print digital format publications designed accompany reading journey clear concise style makes easy understanding providing perfect opportunity improve literary knowledge time br br b see best literature whole new light brightsummaries com b j k rowling mary grandprfantasy fiction young adult')

'harri potter half blood princ j k rowl book analysi detail summari analysi read guid b unlock straightforward side i harri potter half blood princ i concis insight summari analysi b br br thi engag summari present analysi i harri potter half blood princ i j k rowl follow harri tri learn much possibl lord voldemort power weak still keep schoolwork deal teenag drama tri keep eye rival draco malfoy behav oddli ever sinc return hogwart i harri potter half blood princ i sixth novel j k rowl bestsel seven book seri inspir film video game board game even theme park rowl household name around world book includ i the casual vacanc i 2012 i the cuckoo call i 2013 publish pseudonym robert galbraith br br find everyth need know i harri potter half blood princ i in fraction time br br b i thi in depth inform read guid bring you i b br br complet plot summari br charact studi br key theme symbol br question reflect br br i whi choos brightsummari com i br br avail print digit format public design a

In [83]:
df_fix['tags'].apply(stem)

0       harri potter half blood princ j k rowl book an...
1       harri potter order phoenix j k rowl book analy...
2       harri potter sorcer s stone select theme motio...
3       harri potter prison azkaban novel studi gr 4 8...
4       harri potter goblet fire j k rowl book analysi...
                              ...                        
6197    16 person type nutshel b meet self suffici bri...
6198    talk fast can gilmor girl gilmor girl and ever...
6200    underground railroad winner pulitz prize ficti...
6203    love warrior memoir b 1 i new york time i best...
6204    hate u give printz honor winner8 star review g...
Name: tags, Length: 5797, dtype: object

In [84]:
df_fix['tags'] = df_fix['tags'].apply(stem)

<ipython-input-84-f3c16866a5ec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fix['tags'] = df_fix['tags'].apply(stem)


In [85]:
from sklearn.feature_extraction.text import CountVectorizer
cv  = CountVectorizer(max_features=6000 , stop_words= 'english')

cv.fit_transform(df_fix['tags']).toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [86]:
vectors = cv.fit_transform(df_fix['tags']).toarray()
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [87]:
cv.get_feature_names_out()

array(['000', '10', '100', ..., 'zombi', 'zone', 'zoo'], dtype=object)

In [88]:
vectors.shape

(5797, 6000)

In [89]:
import tensorflow as tf
vectors = vectors.astype(np.float32)  # Mengonversi ke tipe data float32

vectors_norm = tf.linalg.norm(vectors, axis=1, keepdims=True)
similarity = tf.matmul(vectors, vectors, transpose_b=True) / (vectors_norm * vectors_norm)

In [90]:
similarity = similarity.numpy()

In [91]:
similarity

array([[1.        , 0.89193547, 0.29354838, ..., 0.816129  , 0.43709677,
        0.08387097],
       [0.8366111 , 0.9999999 , 0.28744325, ..., 0.8108925 , 0.4447806 ,
        0.07866868],
       [1.2999998 , 1.3571427 , 0.9999999 , ..., 1.5857141 , 0.8285713 ,
        0.08571427],
       ...,
       [0.4223706 , 0.44741234, 0.18530884, ..., 1.        , 0.3806344 ,
        0.09599333],
       [0.5634095 , 0.61122656, 0.24116422, ..., 0.94802487, 0.99999994,
        0.13305612],
       [0.23636365, 0.23636365, 0.05454546, ..., 0.5227273 , 0.2909091 ,
        1.0000001 ]], dtype=float32)

In [92]:
similarity.shape

(5797, 5797)

In [93]:
similarity[0]

array([1.        , 0.89193547, 0.29354838, ..., 0.816129  , 0.43709677,
       0.08387097], dtype=float32)

In [94]:
similarity[1]

array([0.8366111 , 0.9999999 , 0.28744325, ..., 0.8108925 , 0.4447806 ,
       0.07866868], dtype=float32)

In [95]:
# def recommend(book):
#     index = np.where(df_fix['books_id'] == book)[0][0]
#     similar_books = sorted(enumerate(similarity[index]) , key=lambda x:x[1]   , reverse=True)[1:11]

#     for i in similar_books:
#         print (df_fix['books_id'][i[0]])
def recommend(book1, book2, book3):
    books = [book1, book2, book3]
    similarity_combined = np.zeros(len(df_fix))  # Inisialisasi similarity_combined dengan array nol

    for book in books:
        index = np.where(df_fix['books_id'] == book)[0]
        if len(index) > 0:
            index = index[0]
            similarity_combined += similarity[index]  # Menambahkan similarity dari setiap buku

    similar_books = sorted(enumerate(similarity_combined), key=lambda x: x[1], reverse=True)[1:11]
    for book in similar_books:
        print(df_fix['books_id'][book[0]])

In [96]:
recommend(1, 6670, 7926)

11574
7257982
49465
49251
8765461
61329
11275
119
1914973
7039218


In [97]:
import h5py
from sklearn.preprocessing import StandardScaler

# Hapus kolom dengan nilai string dari DataFrame
df_numeric = df_fix.select_dtypes(include=[np.number])

# Skalakan DataFrame
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_numeric)

# Simpan DataFrame dan similarity matrix ke dalam file H5
with h5py.File('model.h5', 'w') as hf:
    hf.create_dataset('df_scaled', data=df_scaled)
    hf.create_dataset('similarity', data=similarity)